In [ ]:
import matplotlib.pyplot  as plt 
import pandas as pd
import numpy as np
from numpy import linalg as lin
 
df = pd.read_csv("imports-85.csv", header= None)
df.columns = ['symboling','losses','make','fuel-type','aspiration','doors','body-style','drive-wheels','engine-location','wheel-base','length', 'width','height','curb-weight','engine-type',
              'cylinders','engine-size','fuel-system','bore','stroke','ratio','horsepower','peak-rpm' ,'city-mpg' ,'highway-mpg','price']

FileNotFoundError: ignored

Preprocessing of data

In [ ]:
#for one hot encoding
def dummy(df,col):
    dum = pd.get_dummies(df[col])
    if(len(dum.columns)==2):
        df[col] = dum[dum.columns[1]]
    else:
        df.drop(columns=col)
        df = pd.concat([df, dum], axis=1)
    return df

In [ ]:
category = ['make','fuel-type','aspiration','doors','body-style','drive-wheels','engine-location','engine-type','cylinders','fuel-system']
for i in category:
    df = dummy(df,i)
    df = df.drop(columns=i)
#replace empty spaces
df.replace('?', np.nan, inplace=True)
df.dropna(axis=0, thresh=2, inplace=True)
#replace the empty spaces by average
avg = ['losses','bore','stroke','horsepower','peak-rpm','price']
for i in avg:
    df[i]=pd.to_numeric(df[i],errors='coerce')
    df[i] = df[i].replace(np.nan,df[i].mean())
price = df['price']
df = df.drop(columns='price')
df['price']=price
arr = df.to_numpy

NameError: ignored

In [ ]:
#shuffle data set
shuffle= [163, 181,  48, 109, 119, 134, 155, 113,  42,  56, 144, 107,  11,
        72,  44, 161, 168,   3, 130,  62,  58, 171,  64, 202,  73, 149,
        95,  57, 191, 183,   8, 188,  82, 152, 169,  76,  69, 106,  49,
       200, 203,  89, 105,  96,  70,  68,  59, 122, 182,  26, 201, 102,
        28,  25, 118,  94,  41,  83, 136, 172, 112, 187, 143, 100, 194,
        27, 117,  20, 175,  53, 192,   9, 138, 176, 139,  47, 153,  86,
        91, 185,   0,  18, 124, 166, 173, 156,  34,   7, 108, 160, 151,
        99,  77,  78,  40,  67, 170, 162, 179,  17, 180, 111, 167,  75,
        74,  87, 114, 159, 184,  80, 147, 137, 101, 131,   1,  93,  16,
        39,  36, 165, 127,  37, 197, 120,  52, 125,  79,  43,  31, 154,
        21,  98,  46, 116, 148,  51,  13, 178, 190,  33,  92, 174, 186,
        55,  19,  12,  24,  38, 135,  66,  90, 128, 196, 129, 198,   5,
        84, 199,  10,  81,  14,  63,  88, 146, 145,  85,  15, 123,   6,
        50,  60, 132,  54, 140, 177, 115,  65, 157, 164,   2, 150, 104,
       204, 158,  97,  23, 195, 193,   4, 103,  32, 142, 126,  71, 189,
       141,  45, 121,  22,  35,  61,  30, 133,  29, 110]


randdf = df.reindex(shuffle)
arr = randdf.to_numpy() 
testno = int(len(arr)*0.781)
train = arr[:testno]
test = arr[testno:,:-1]
testpred  = arr[testno:,-1]
#valid = int(len(testno)*0.6)
#test = arr[:valid]
#valid = arr[valid:]


In [ ]:
#Train data with last column as price
#Test data with no last colum
#pred contains test data actual value

def KNN(X,l,n):
    ldist = []
    ldist.clear()
    lp=[]
    for j in range(0,len(X)):
        ldist.clear()
        for i in range(0,len(l)):
            dist = X[j] - l[i][:-1]
            ldist.append((lin.norm(dist,ord=1),l[i][-1]))
        ldist.sort() 
        sum = 0
        for i in range(0,n):
            sum = ldist[i][1] + sum
        lp.append(sum/n)
    return lp

In [ ]:
def err(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

Split training dataset for cross validation

In [ ]:
def split_given_size(a, size):
    return np.split(a, np.arange(size,len(a),size))
c = split_given_size(train,32)

In [ ]:
rval = c[1]
rtrain = (np.concatenate(c[:1]+c[2:]))
rpred = rval[:,-1]
rval = rval[:,:-1]
pred = KNN(rval,rtrain,2)
sum = err(pred,rpred)
print(sum)


TypeError: ignored

The RSME is 3606.6

In [ ]:
#cross validation for optimal value of k
rerror = []
rsme = 0
for n in range(1,30):
    rsme = 0
    for i in range(5):
        rval = c[i]
        rtrain = (np.concatenate(c[:i]+c[i+1:]))
        rpred = rval[:,-1]
        rval = rval[:,:-1]
        pred = KNN(rval,rtrain,n)
        sum = err(pred,rpred)
        rsme = rsme + sum
    rerror.append((rsme/5,n))

The optimal value of k =5

In [ ]:
a,b =zip(*rerror)
plt.scatter(b,a)
plt.show()

To select top features

In [ ]:
#average rsme for all featues only single feautre is compared with the predicted value
k = []
for i in range(70):
    ftrain = train[:,[i,-1]]
    ftest = test[:,[i]]
    fy=testpred[:]
    pred = KNN(ftest,ftrain,n) 
    rsme = err(pred,fy)
    k.append((rsme,df.columns[i]))
k.sort()
print(k)

Top 8 features are selected to reduce the rsme


In [ ]:
newdf =df[['engine-size','city-mpg','curb-weight','highway-mpg','horsepower','wheel-base','width','length','price']]
randdf = newdf.reindex(shuffle)
arr = randdf.to_numpy() 
ptestno = int(len(arr)*0.781)
ptrain = arr[:testno]
ptest = arr[testno:,:-1]
testpred  = arr[testno:,-1]
c = split_given_size(ptrain,32)

In [ ]:
regerror = []
rsme = 0
for i in range(1,30):
    rsme=0
    for j in range(5):
        nrval = c[j]
        nrtrain = (np.concatenate(c[:j]+c[j+1:]))
        nrpred = nrval[:,-1]
        nrval = nrval[:,:-1]
        pred = KNN(nrval,nrtrain,i)
        sum = err(pred,nrpred)
        rsme = rsme + sum
    regerror.append((rsme/5,i))

The optimal value of K changes from 5 to 7 when highly related features are taken.
The number of feature variable were reduced and features relevant to model were take in count to make model more generalized

In [ ]:
a,b =zip(*regerror)
plt.scatter(b,a)
plt.show()
regerror.sort()

In [ ]:
regerror

**Q2 Decision Tree**

In [ ]:
df = pd.read_csv("adult.csv", header=None)
df.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','pred']


In [ ]:
#replace and drop empty spaces
df.replace(' ?', np.nan,inplace=True)
df.dropna(inplace=True)

head = ['workclass','marital-status','occupation','relationship','race','sex','native-country','pred']
#convert into label encoding
for i in head:
   a = df[i].to_numpy() 
   a = np.apply_along_axis(np.unique, 0, a, return_inverse=True)
   df[i]=a[1]
df.drop(columns='education')

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,pred
0,39,5,77516,13,4,0,1,4,1,2174,0,40,38,0
1,50,4,83311,13,2,3,0,4,1,0,0,13,38,0
2,38,2,215646,9,0,5,1,4,1,0,0,40,38,0
3,53,2,234721,7,2,5,0,2,1,0,0,40,38,0
4,28,2,338409,13,2,9,5,2,0,0,0,40,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,2,257302,12,2,12,5,4,0,0,0,38,38,0
32557,40,2,154374,9,2,6,0,4,1,0,0,40,38,1
32558,58,2,151910,9,6,0,4,4,0,0,0,40,38,0
32559,22,2,201490,9,4,0,3,4,1,0,0,20,38,0


In [ ]:
#The below features were converted into interval of that feature
for i in ['hours-per-week','age']:
    label={}
    label['a'] = df[i]
    label['a'] = pd.cut(df[i], 10)
    a = label['a'].to_numpy() 
    a = np.apply_along_axis(np.unique, 0, a, return_inverse=True)
    label['a']=a[1]
    df[i] = label['a']

#continious features are dropped
fd = df.drop(columns=['capital-gain','capital-loss','fnlwgt','education',])

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
class Node():
    def __init__ (self,depth = 0, parent = None,):
        self.depth = depth
        self.parent = parent
        self.child = {}
        self.right = None
        self.leaf = True
        self.label = 0
        self.current = None

    
    #calculate the entropy   
    def entropy(self,df,feature):
        a = len(df[df[feature]==1])
        b = len(df[df[feature]==0])
        ent = 0
        if(a!=0):
            ent =ent  -(a/(a+b))*np.log2(a/(a+b))
        if(b!=0):
            ent =ent - (b/(a+b))*np.log2(b/(a+b))
        return ent
    
    #calculate the information gain
    def infogain(self,df, feature):
        l = np.unique(df[feature])
        infogain = self.entropy(df,'pred')
        b = len(df[feature])
        for i in l:
            a = len(df[df[feature]==i])
            infogain = infogain-(a/b)*self.entropy(df[df[feature]==i],'pred')
        return infogain
    
    #predict the output
    def predict(self,df):
        if self.leaf:
            return self.label
        else:
            a = df[self.current]
            try : 
                c = (self.child[a].predict(df))
            except:
                c = self.label
            return c

    #for the best feature selection
    def sel(self,df,feature):
        maxinfo = -1
        maxfeature = None
        for i in feature:
            a = self.infogain(df,i)
            if a > maxinfo:
                maxinfo = a
                maxfeature=i
        return maxfeature

    #train the tree
    def train(self,feature,df,leafsize,depth):
        if len(df) < leafsize or self.depth >= depth:
            self.leaf = True
            a=len(df[df['pred']==0])
            b=len(df[df['pred']==1])
            if(b>a):
                self.label=1
            else:
                self.label=0
        else:
            ch = {}
            self.leaf=False
            self.current=self.sel(df,feature)
            newlist = feature[:]
            newlist.remove(self.current)
            l = np.unique(df[self.current])
            for i in l:
                ch[i] = Node(depth=self.depth+1 , parent=self)
            for i in l:
                ch[i].train(newlist,df[df[self.current]==i] , leafsize , depth)
            self.child=ch

    

In [ ]:
class Tree:
    def __init__(self,leafsize = 10, depth = 5 ):
        self.root = Node()
        self.leafsize = leafsize
        self.depth = depth 

    def predict( self, df ):
        return self.root.predict(df )

    def train(self,df,feature):
        self.root.train(feature,df,self.leafsize,self.depth )

In [ ]:
feat = list(fd.columns)
feat.remove('pred')
#accuracy of the decision tree
def accuracy(a,b):
    c = (a==b).value_counts()
    acc = (c[True]/(c[True]+c[False]))*100
    return acc

In [ ]:
text = int(len(fd)*0.7)
train = fd.iloc[:text]
test = fd.iloc[text:]
rtest =train

In [ ]:
#initialized the tree
DT = Tree(leafsize=400,depth=7)
part = np.array_split(rtest,10)
for i in range(0,len(part)):
    pd.DataFrame(part[i])
rtrain = pd.concat(part[0:8])
DT.train(rtrain,feat)
rest = part[9]

In [ ]:
p =[]
for i in range(len(rest)):
    p.append(DT.predict(rest.iloc[i]))
pd.DataFrame(p)
acc = accuracy(p[0],rest['pred'])
print(acc)

75.08289909995263


Accuracy on initial validation set is 75.1

In [ ]:
#cross validation for parameter optimization
l = []
part = np.array_split(rtest,5)
for z in range(0,len(part)):
    pd.DataFrame(part[z])
for i in range(50,750,100):
    for j in range(1,7):
        DT = Tree(leafsize=i,depth=j)  
        sum = 0    
        for z in range(len(part)):
            rest = part[z]
            rtrain = pd.concat(part[0:z]+part[z+1:])
            DT.train(rtrain,feat)
            p =[]
            for z in range(len(rest)):
                p.append(DT.predict(rest.iloc[z]))
                pd.DataFrame(p)
                acc = accuracy(p[0],rest['pred'])
            sum=sum+acc
        l.append((sum/5,i,j))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-82-816d1b1c1b46>", line 17, in <module>
    pd.DataFrame(p)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py", line 523, in __init__
    mgr = init_ndarray(data, index, columns, dtype=dtype, copy=copy)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py", line 190, in init_ndarray
    values = _prep_ndarray(values, copy=copy)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py", line 310, in _prep_ndarray
    values = convert(values)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py", line 298, in convert
    return maybe_convert_platform(v)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py", line 95, in maybe_convert_platform
    

KeyboardInterrupt: ignored